In [2]:
import numpy as np
import pandas as pd
import torch
import sys
import models
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
### attempt at recreating the DT-14 dataset from the taxonomy paper 
def prune_leaf(graph):
    ## graph is a directed graph
    leaf_nodes = [node for node in graph.nodes() if graph.in_degree(node)!=0 and graph.out_degree(node)==0]
    k=np.random.randint(0,len(leaf_nodes))
    graph.remove_node(leaf_nodes[k])
    return graph

num_nodes=63   
## pick random angles
num_vecs=32
angles=np.random.rand(num_vecs) * np.pi ## number of vectors with random values between 0 and pi, as y positive
sorted_angles=np.sort(angles)

G=nx.full_rary_tree(2, num_nodes,create_using=nx.DiGraph)
## add the correct angles as data to nodes
leaf_nodes = [node for node in G.nodes() if G.in_degree(node)!=0 and G.out_degree(node)==0]
print(leaf_nodes)
for i, angle in enumerate(sorted_angles): 
    G.nodes[leaf_nodes[i]]["angle"]=angle## put the generated angles on the leaves

## for each level above the leaves do the parent calculation and add that angle.
for i in range(num_vecs-2,-1,-1): ## backwards iteration through the layers
    #vector=np.array([(1/2)*(np.cos(G.nodes[2*i+1]["angle"])+np.cos(G.nodes[2*i+2]["angle"])),(1/2)*(np.sin(G.nodes[2*i+1]["angle"])+np.sin(G.nodes[2*i+2]["angle"]))])
    ### do we really need the vector? Just compute mean angle?
    G.nodes[i]["angle"]=1/2*(G.nodes[2*i+1]["angle"]+G.nodes[2*i+2]["angle"])#np.arctan(vector[1]/vector[0]) ### what should this be?
    print("new angle:",G.nodes[i]["angle"])
    print(G.nodes[2*i+1]["angle"])
    print(G.nodes[2*i+2]["angle"])
leaf_nodes = [node for node in G.nodes() if G.in_degree(node)!=0 and G.out_degree(node)==0]
### prune the tree to be the correct number of leaf nodes
while len(leaf_nodes)>=15:
    prune_leaf(G)
    leaf_nodes = [node for node in G.nodes() if G.in_degree(node)!=0 and G.out_degree(node)==0]
# leaf_nodes = [node for node in G.nodes() if G.in_degree(node)!=0 and G.out_degree(node)==0]
# print(leaf_nodes)
# subax2= plt.subplot(121)
# nx.draw(G)
# print(G.nodes.data())    
    



[31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
new angle: 2.9163774645578546
2.8970587316326757
2.9356961974830336
new angle: 2.5856427835937312
2.5373006465617554
2.6339849206257067
new angle: 2.433949735487924
2.4270437633774815
2.440855707598366
new angle: 2.3820110714676814
2.3735952705787096
2.3904268723566533
new angle: 2.2682701460805754
2.2420432660517715
2.294497026109379
new angle: 1.9448751063765348
1.7376088927345095
2.1521413200185604
new angle: 1.5370115416091572
1.4399161988362434
1.6341068843820707
new angle: 1.3664113002285538
1.340374279578911
1.3924483208781966
new angle: 1.2117053352983471
1.15768815165117
1.2657225189455241
new angle: 1.0711856909584214
1.040933354544549
1.1014380273722937
new angle: 0.9788991335614887
0.9565449063075524
1.001253360815425
new angle: 0.8865692599987511
0.8719769391838349
0.9011615808136675
new angle: 0.7269500113429936
0.7034663501010223
0.750433672584

In [5]:
## generate the actual data from the structure    
    
def generate_data(G, num_data):
# here we generate gaussian data around the generated structure for each of the leaves
# positive and negative samples randomly generated
    leaf_nodes = [node for node in G.nodes() if G.in_degree(node)!=0 and G.out_degree(node)==0]
    data=[]
    res=[]
    labels=[]
    for leaf in leaf_nodes:
        # gen the data
        theta=G.nodes[leaf]["angle"]
        vector=np.array([(theta/np.pi)*np.cos(theta),(theta/np.pi)*np.sin(theta)])
        for i in range(num_data):
            if np.random.rand() >0.5:
                data.append(np.random.multivariate_normal(vector, np.eye(2)))
                labels.append(1)
            else:
                data.append(np.random.multivariate_normal(-vector, np.eye(2)))
                labels.append(0)
        res.append(data)
    return data, labels

data,labels = generate_data(G,100)
#print(np.sum(labels))



In [6]:
### Generate the distance matrix A for the entire tree/graph
A=dict(nx.all_pairs_dijkstra_path_length(G.to_undirected()))
#print(A)
num_nodes_postprune=60 ### how to do this well? We lose the index to node relationship if we remove all empty rows and columns... Maybe there is some way to retain it?
mat=np.zeros((num_nodes_postprune,num_nodes_postprune)) ## only relevant for leaves (?)
for key, value in A.items():
    for key2,value2 in value.items():
        mat[key][key2]=value2
print(mat) 


# #### similarity matrix is not PD.  
# Amax=mat.max()
# tmp=mat/Amax
# print(tmp)
# S=np.ones((num_nodes_postprune,num_nodes_postprune))-tmp
# print(S) 

# # remove all the rows and columns with ones
# S_new=S[~np.all(S == 1, axis=1)]
# S_new=S_new[:,~np.all(S_new == 1, axis=0)]
# print(S_new.min())

#C=np.linalg.cholesky(S_new) ## we cannot do this as we have 0's in the matrix
#loop through the dictionaries and just fill in the values in the matrix
#def generate_distance_matrix():
    # takes graph and returns matrix which holds distances between nodes

### maybe try S_ij= 1- A_ij/A_max
# and have z be the cholesky decomposition of S and then learn with this
# can we recreate y from X and z using some simple FCN?

[[0. 1. 1. ... 0. 0. 5.]
 [1. 0. 2. ... 0. 0. 6.]
 [1. 2. 0. ... 0. 0. 4.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [5. 6. 4. ... 0. 0. 0.]]


In [7]:
### train a model on a subset of datasets/domains and test on the rest

### 
print(leaf_nodes)
print(np.array(data))

## create a matrix for just the leaf nodes and not the entire tree
domain_A=np.zeros((14,14))
for i,node1 in enumerate(leaf_nodes):
    for j,node2 in enumerate(leaf_nodes):
        domain_A[i][j]=mat[node1][node2]
        
indices=[0,1,2,3,4,5,6,7,8,9,10,11,12,13]
domain_index=[i*np.ones(100) for i in indices]
#print(np.reshape(np.array(domain_index),-1)) The elements are float. May be an issue, or maybe not..
source_data=np.array(data)[:399] # first 4 leaves/domains
target_data=np.array(data)[400:]

source_indices=domain_index[:399]
target_indices=domain_index[400:]

source_labels=labels[:399]
target_labels=labels[400:]

#model=models.BaseModel()


## raw data encoder x_l->h_l

## taxonomy encoder, A,u_l -> z_u_l

## joint encoder, h_l,z_u_l -> e_l; final embedding

# For fair comparison, all baselines and TSDA use the same encoder and predictor. 
# The encoder has the following components:
# • A raw data encoder embeds the data xl into intermediate embeddings hl
# • A taxonomy encoder embeds the domain distance matrix A and the domain index ul to the domain embeddings zul
# with 1 fully connected (FC) layer. We use a taxonomy embedding loss Lg to pretrain the taxonomy encoder:
# Lg = Eu1,u2∼p(u)# [lg(∥z⊤u1 zu2∥2, Au1,u2 )],
# where u1, u2 are two independent domain identities sampled from p(u), and lg denotes a regression loss (e.g.,ℓ2
# distance).
# • A joint encoder then takes as input both hl and zl and produces the final embeddings el with 2 FC layers.
# For toy datasets DT-14 and DT-40, we use 3 FC layers as the raw data encoder, 
# while for the real dataset ImageNet-AttributeDT and CUB-DT, 
# we use PyTorch’s default pretrained Resnet-18 as the raw data encoder.
# All the predictors of baselines and TSDA contain 3 FC layers, and all the discriminators have 6 FC layers.
# For GRDA, we treat every pair of domains that share a common grandparent node as connected, 
# construct GRDA’s domain graph, and feed it into its discriminator to recover the graph.
# For the structure of the taxonomist, we first use a 6-FC-layer neural network T to produce
# a 2-dimensional taxonomy representation tl of the data embedding el, 
# and then calculate the ℓ2 distance of a pair of taxonomy representations tl.


# All the predictors of baselines and TSDA contain 3 FC layers, and all the discriminators have 6 FC layers. 
# For GRDA, we treat every pair of domains that share a common grandparent node as connected,
# construct GRDA’s domain graph, and feed it into its discriminator to recover the graph.

# For the structure of the taxonomist, we first use a 6-FC-layer neural network T
# to produce a 2-dimensional taxonomy representation tl of the data embedding el 
# and then calculate the ℓ2 distance of a pair of taxonomy representations tl




[15, 16, 19, 20, 21, 22, 24, 35, 37, 48, 52, 53, 56, 59]
[[-0.93940484  0.03541042]
 [-0.32006535  1.09701517]
 [ 0.85903057  0.83661454]
 ...
 [ 0.89112729  2.16843475]
 [-0.55875117  1.45407259]
 [ 0.33080101 -0.24379352]]


In [ ]:
### Make adjacency graph

# A=np.zeros((63,63))
# for i,_ in enumerate(A): incorrect, do it right!
#     if i<=31:
#        
# print(A)
# num_nodes=7
#def make_graph(num_nodes, angle_all):
# A = np.zeros((num_nodes, num_nodes))
# for i in range(num_nodes//2):
#         #for j in range(i + 1, num_nodes):
#             #p = np.cos(angle_all[i]) * np.cos(angle_all[j]) + np.sin(angle_all[i]) * np.sin(angle_all[j])
#             # for each node we should have that it has the desired connections, it is a binary tree!
            
#         ## both children, i.e. i has 2*i and 2*i+1 as children, add 1 for the index starting at 0
#         A[i][(2*i)+1] = 1
#         A[i][(2*i+1)+1] = 1
#         A[(2*i+1)+1][i] = 1
#         A[(2*i)+1][i] = 1
    
# print(A)


# ## create the parents for each group of vectors
# def make_parents(sorted_angles):
#     a=sorted_angles[::2] 
#     parents=np.zeros((len(a),2))
#     for i,angle in enumerate(a):
#         ## create parent vector
#         parents[i]=(1/2)*np.array([np.cos(sorted_angles[2*i])+np.cos(sorted_angles[2*i]),np.sin(sorted_angles[2*i])+np.sin(sorted_angles[2*i])])
#     #print(parents)
#     print(np.arctan(parents[:,0]/parents[:,1]))
#     return np.arctan(parents[:,0]/parents[:,1]) #parents angles return

# ## append the entire structure to an array, should we do this?
# structure=[]
# structure.append(sorted_angles)
# k=2
# tmp_angles=sorted_angles
# while k!=1:
#     parents=make_parents(tmp_angles)
#     k=len(parents)
    
#     tmp_angles=parents
#     print(tmp_angles)
#     structure.append(parents)
# print(structure)



In [ ]:
### network which takes in data, some additional structure data start with an index or similar

### reconstruction loss of structure data

## classification head

## encoder/ discriminator a reasonable option for this?

### maybe try S_ij= 1- A_ij
# and have z be the cholesky decomposition of S and then learn with this
# can we recreate y from X and z using some simple FCN?